In [1]:
from queue import PriorityQueue

# Q1

In [5]:
class Node:
    def __init__(self, position, parent=None, visited_goals=set()):
        self.position = position
        self.parent = parent
        self.visited_goals = visited_goals.copy()
        self.g = 0
        self.h = 0
        self.f = 0

    def __lt__(self, other):
        return self.f < other.f

def heuristic(pos, goal):
    return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])

def multi_goal_best_first_search(maze, start, goals):
    rows, cols = len(maze), len(maze[0])
    start_node = Node(start, visited_goals=set())

    frontier = PriorityQueue()
    frontier.put(start_node)
    visited_states = {}

    while not frontier.empty():
        current_node = frontier.get()
        current_pos = current_node.position
        current_goals_visited = current_node.visited_goals

        if current_goals_visited == goals:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)
            if 0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and maze[new_pos[0]][new_pos[1]] == 0:
                new_visited_goals = current_goals_visited.copy()
                if new_pos in goals:
                    new_visited_goals.add(new_pos)

                new_node = Node(new_pos, current_node, new_visited_goals)
                new_node.g = current_node.g + 1
                new_node.h = sum(heuristic(new_pos, g) for g in goals - new_visited_goals)
                new_node.f = new_node.g + new_node.h

                state_key = (new_pos, frozenset(new_visited_goals))
                if state_key not in visited_states or new_node.g < visited_states[state_key]:
                    visited_states[state_key] = new_node.g
                    frontier.put(new_node)
    return None


# Q2

In [8]:
import heapq
import random
import time
import threading

class DynamicGraph:
    def __init__(self, graph):
        self.graph = graph 
        self.lock = threading.Lock()

    def get_neighbors(self, node):
        with self.lock:
            return self.graph.get(node, [])

    def update_edge_costs(self):
        while True:
            time.sleep(random.randint(3, 6)) 
            with self.lock:
                node = random.choice(list(self.graph.keys()))
                if self.graph[node]:
                    idx = random.randint(0, len(self.graph[node]) - 1)
                    neighbor, old_cost = self.graph[node][idx]
                    new_cost = random.randint(1, 15)
                    self.graph[node][idx] = (neighbor, new_cost)
                    print(f"\nEdge cost updated: {node} -> {neighbor} | {old_cost} → {new_cost}\n")

def heuristic(a, b):
    return abs(ord(a) - ord(b))

def a_star(graph, start, goal):
    frontier = []
    heapq.heappush(frontier, (0, start))
    g_cost = {start: 0}
    came_from = {start: None}

    while frontier:
        _, current = heapq.heappop(frontier)

        if current == goal:
            path = []
            while current:
                path.append(current)
                current = came_from[current]
            path.reverse()
            print(f"\nOptimal Path: {path} | Cost: {g_cost[goal]}")
            return path

        for neighbor, cost in graph.get_neighbors(current):
            new_g = g_cost[current] + cost
            if neighbor not in g_cost or new_g < g_cost[neighbor]:
                g_cost[neighbor] = new_g
                f_value = new_g + heuristic(neighbor, goal)
                heapq.heappush(frontier, (f_value, neighbor))
                came_from[neighbor] = current

    print("No path found.")
    return None

graph_data = {
    'A': [('B', 5), ('C', 8)],
    'B': [('D', 10), ('E', 3)],
    'C': [('F', 2)],
    'D': [('G', 7)],
    'E': [('G', 5)],
    'F': [('G', 6)],
    'G': []
}

dynamic_graph = DynamicGraph(graph_data)

cost_update_thread = threading.Thread(target=dynamic_graph.update_edge_costs, daemon=True)
cost_update_thread.start()

a_star(dynamic_graph, 'A', 'G')



Optimal Path: ['A', 'B', 'E', 'G'] | Cost: 13


['A', 'B', 'E', 'G']


Edge cost updated: B -> E | 3 → 12


Edge cost updated: F -> G | 6 → 3


Edge cost updated: D -> G | 7 → 4


Edge cost updated: B -> D | 10 → 14


Edge cost updated: C -> F | 2 → 13


Edge cost updated: F -> G | 3 → 4


Edge cost updated: A -> C | 8 → 10


Edge cost updated: E -> G | 5 → 12


Edge cost updated: A -> C | 10 → 5


Edge cost updated: B -> D | 14 → 13


Edge cost updated: E -> G | 12 → 4



# Q3

In [1]:
import heapq
import random

class DeliveryPoint:
    def __init__(self, name, location, time_window):
        self.name = name
        self.location = location 
        self.time_window = time_window 

    def __repr__(self):
        return f"{self.name}({self.time_window})"

def heuristic(current_location, target_location):
    return abs(current_location[0] - target_location[0]) + abs(current_location[1] - target_location[1])

def greedy_best_first_search(start, delivery_points):
    frontier = []
    heapq.heappush(frontier, (0, start, [], 0)) 

    visited = set()

    while frontier:
        _, current_location, path, current_time = heapq.heappop(frontier)

        if len(path) == len(delivery_points):  
            print(f"Optimized Route: {path} | Total Time: {current_time}")
            return path

        remaining_deliveries = sorted(
            [(dp, heuristic(current_location, dp.location)) for dp in delivery_points if dp not in path],
            key=lambda x: (x[0].time_window[1], x[1])  
        )

        for delivery, distance in remaining_deliveries:
            travel_time = random.randint(5, 15)  

            arrival_time = current_time + travel_time
            if arrival_time > delivery.time_window[1]:  
                continue  

            new_path = path + [delivery]
            heapq.heappush(frontier, (distance, delivery.location, new_path, arrival_time))

    print("No feasible delivery route found.")
    return None

deliveries = [
    DeliveryPoint("D1", (2, 3), (8, 10)),
    DeliveryPoint("D2", (5, 6), (9, 12)), 
    DeliveryPoint("D3", (1, 4), (7, 11)), 
]

start_location = (0, 0)
optimized_route = greedy_best_first_search(start_location, deliveries)


No feasible delivery route found.
